In [10]:
import time
import cv2 as cv
import numpy as np
import pyautogui
import torch
import os
import yaml
import shutil
from mss.linux import MSS as mss
from ultralytics import YOLO
from datasets import load_dataset
from pathlib import Path

In [7]:
# loading the dataset from huggingface that is in COCO (Common Objects in Context) format
ds = load_dataset("keremberke/csgo-object-detection", name="full")

# start training from scratch
model_9m = YOLO("yolov9m.pt").to("cuda")
model_9s = YOLO("yolov9s.pt").to("cuda")

In [8]:
def make_folders(root, img_lab, train_val_test):
    needed_path = root / img_lab / train_val_test
    needed_path.mkdir(parents = True, exist_ok=True)
    
# writing a function that extracts the labels for each image in the dataset in YOLO format
def save_img_label(df, train_val_test, output_dir, rounding = 6):
    
    output_dir_image = output_dir / "images"
    output_dir_label = output_dir / "labels"
    
    # ensure the output directory exists
    os.makedirs(output_dir_image, exist_ok = True)
    os.makedirs(output_dir_label, exist_ok = True)
    
    for index, item in enumerate(df[train_val_test]):
        current_obj = item['objects']
        label = ""
        for i, _ in enumerate(current_obj['category']):
            name = current_obj['category'][i]
            # the width and height of each picture is known to be 416 x 416, we need this to normalize everything
            bbox = [np.round(x/416, rounding) for x in current_obj['bbox'][i]]
            # bbox is in (xmin, ymin, width, height) format, we need to convert it to (xcenter, ycenter, width, height) format
            xcenter = np.round(bbox[0] + bbox[2]/2, rounding)
            ycenter = np.round(bbox[1] + bbox[3]/2, rounding)
            
            if name in [1,3]:
                label = label
            else: 
            # now we create the label
                label += f"0 {xcenter} {ycenter} {bbox[2]} {bbox[3]} \n"
            
        img = item['image']

        output_path_label = os.path.join(output_dir_label / train_val_test, f"{index}.txt")
        output_path_image = os.path.join(output_dir_image / train_val_test, f"{index}.JPG")
        
        
        with open(output_path_label, 'w') as f:
            f.write(label)
            
        img.save(output_path_image)   

In [9]:
root_path = Path("yolo_data")

for i in ["images", "labels"]:
    for j in ["train", "validation", "test"]:
        make_folders(root_path, i, j)
        
for i in ["train", "validation", "test"]:
    save_img_label(ds, i, root_path)

In [16]:
temp2 = model_9s.train(data = "config.yaml", epochs = 200)

New https://pypi.org/project/ultralytics/8.2.75 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov9s.pt, data=config.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

c:\Python\Lib\site-packages\ultralytics\engine\trainer.py:269: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)


In [141]:
# now we try to process all the datasets such that they are all only detecting other players. 
# essentially, we delete all the head categories, and force c and ct to be one group

def process_line(line, delete_numbers, replace_map):
    # Split the line by spaces
    parts = line.strip().split()
    if len(parts) > 0:
        parts = [float(item) for item in parts]
        # If the first value is in the delete list, return None (to delete the line)
        if parts[0] in delete_numbers:
            return None
        
        # If the first value is in the replace_map, replace it with the corresponding value
        if parts[0] in replace_map:
            
            parts[0] = replace_map[parts[0]]
            
        parts[0] = int(parts[0])
        
        parts = [str(item) for item in parts]
    
    # Return the modified line as a string
    return " ".join(parts)

def process_file(file_path, delete_numbers, replace_map):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Process each line in the file
    modified_lines = []
    for line in lines:
        modified_line = process_line(line, delete_numbers, replace_map)
        if modified_line is not None:
            modified_lines.append(modified_line)
    
    # Write the modified lines back to the file
    with open(file_path, 'w') as file:
        file.write("\n".join(modified_lines) + "\n")

In [154]:
# # names: ['CT', 'T'], we shall turn all the 1 into 0
# dir = Path(r"C:\Users\yangs\Desktop\training data")

# delete_numbers = []

# replace_map = {1 : 0}

# for i in ["1","2", "4"]:
#     for j in ["test", "valid", "train"]:
#         temp_dir = dir / i / j / "labels"
#         for filename in os.listdir(temp_dir):
#             if filename.endswith('.txt'):
#                 file_path = os.path.join(temp_dir, filename)
#                 process_file(file_path, delete_numbers, replace_map)
                
# dir = Path(r"C:\Users\yangs\Desktop\training data")
# target_dir = Path(r"C:\Users\yangs\Documents\Python Projects\YOLO-Head-Detection\yolo_data\labels\train")
# for i in ["1","2", "4"]:
#     for j in ["test", "valid", "train"]:
#         temp_dir = dir / i / j / "labels"
#         for txt_file in temp_dir.glob("*.txt"):
#             # Copy the file to the target directory
#             shutil.copy(txt_file, target_dir / txt_file.name)
            
# dir = Path(r"C:\Users\yangs\Desktop\training data")
# target_dir = Path(r"C:\Users\yangs\Documents\Python Projects\YOLO-Head-Detection\yolo_data\images\train")
# for i in ["1","2", "4"]:
#     for j in ["test", "valid", "train"]:
#         temp_dir = dir / i / j / "images"
#         for img_file in temp_dir.glob("*.JPG"):
#             # Copy the file to the target directory
#             shutil.copy(img_file, target_dir / img_file.name)
